In [5]:
import pe_modules
import users
import dataloaders
import llms
import jinja2
import argparse
import yaml

config_path = "./configs/david_base_config.yaml"

config = yaml.safe_load(open(config_path))

# Load data
dataloader_class = dataloaders.DATALOADER_CLASSES[config['data']['data_loader_name']]
dataloader = dataloader_class(config) # Could force to be DataLoader class here?
items = dataloader.get_data()

# Load other helpers
jinja_env = jinja2.Environment(loader=jinja2.FileSystemLoader(searchpath='./templates'))
llm_module = llms.LLM_CLASSES[config['llm']['llm_name']]
llm = llm_module(config)


In [7]:
i = 3

item_desc = items[i]['description'] #NOTE: Currently item_idx is a list to support returning multiple
interactions = [{"query": "Are you looking for a football?", "response": "Yes"}]

template_file = config['llm']['like_probs_template'] 
query_template = jinja_env.get_template(template_file)
context = {
            "item_desc": item_desc,
            "interactions": interactions
}
query = query_template.render(context)
print(query)


You must determine whether or not a user will like an item based on a description of the item and 
a sequence of queries posed to the user and the user's responses. Your answer can only be true or false.

''''
Example 1:
A user has the following past interactions:
####
Query: Do you care about fuel mileage? 
Response: No, I don't really care about that.
####
####
Query: Do you want a sports car? 
Response: Yes
####
An item has the following descriptions: 
####
Description: This is a grey four-door sedan that focuses on fuel efficiency and low budget.
####
Is the following statement true or false: This user will like this item. Answer only with True or False.
Your response: False
''''

''''
Example 2:
A user has the following past interactions:
####
Query: Do you want an electric guitar or an acoustic guitar?
Response: Electric
####
####
Query: What type of pickups do you want?
Response: Two humbucker pickups
####
####
Query: Is there a brand that you prefer?
Response: I like Gibson, b

In [9]:
response = llm.make_request(query, temperature=config['llm']['temperature'], logprobs=1)
print(response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": {
        "text_offset": [
          1626
        ],
        "token_logprobs": [
          -0.6548007
        ],
        "tokens": [
          "True"
        ],
        "top_logprobs": [
          {
            "True": -0.6548007
          }
        ]
      },
      "text": "True"
    }
  ],
  "created": 1701962076,
  "id": "cmpl-8TANAvcAEv4vnJe81heAgXqPuq74w",
  "model": "gpt-3.5-turbo-instruct",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 1,
    "prompt_tokens": 364,
    "total_tokens": 365
  }
}


In [20]:
print(response['choices'][0]["logprobs"]["token_logprobs"])


{
  "True": -0.6548007
}
